In [1]:
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 7)
import pyspark

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import pyspark.pandas as ps

import os
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'


In [2]:
from pyspark.sql import SparkSession

# create a SparkSession
spark = SparkSession.builder.appName("PostgreSQL").getOrCreate()


# Set database connection properties
host="host.docker.internal"
database="data6300"
user="aqidb"
password="mypassword"
port="5432"

# Create a JDBC URL
url = f"jdbc:postgresql://{host}:{port}/{database}"

# Set the connection properties
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/28 23:00:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# load data from a PostgreSQL table into a PySpark DataFrame
df = spark.read.jdbc(url=url, table="historic_gas_info", properties=properties)

# display the data
df.show(20)

+----------+------+----+----+-----+----+-----+----+----+---+-----------+--------+---------+
|      date|    co|  no| no2|   o3| so2|pm2_5|pm10| nh3|aqi|        lon|     lat|coord_key|
+----------+------+----+----+-----+----+-----+----+----+---+-----------+--------+---------+
|2022-10-10|173.57| 0.0|1.48|55.79|0.07| 1.19|1.26|2.41|1.0|-102.462776|51.21389|        1|
|2022-10-10|173.57| 0.0|1.61|52.93|0.07| 1.19|1.26|2.31|1.0|-102.462776|51.21389|        1|
|2022-10-10|173.57| 0.0|1.74|49.35|0.06| 1.27|1.34|1.96|1.0|-102.462776|51.21389|        1|
|2022-10-10|173.57| 0.0|1.76|46.49|0.07|  1.5|1.58|1.52|1.0|-102.462776|51.21389|        1|
|2022-10-10|175.24| 0.0|1.69|44.35|0.12|  2.0| 2.1|1.16|1.0|-102.462776|51.21389|        1|
|2022-10-10|178.58| 0.0|1.69|42.92|0.19| 2.71|2.84|0.89|1.0|-102.462776|51.21389|        1|
|2022-10-10|183.58| 0.0|1.82|41.13| 0.2| 3.37|3.53|0.75|1.0|-102.462776|51.21389|        1|
|2022-10-10|185.25| 0.0|2.01|38.62|0.16| 3.87|4.05|0.71|1.0|-102.462776|51.21389

In [ ]:
df = df.drop(columns='Unnamed: 0')
df.

,dt,co,no,no2,o3,so2,pm2_5,pm10,nh3,aqi,lon,lat
0,2023-01-01 05:00:00,293.73,0.00,7.88,46.49,1.27,3.69,4.70,6.08,1,-102.4628,51.2139
1,2023-01-01 06:00:00,300.41,0.00,7.80,44.70,1.09,3.94,5.05,6.46,1,-102.4628,51.2139
2,2023-01-01 07:00:00,297.07,0.00,7.54,42.20,0.81,4.33,5.57,7.03,1,-102.4628,51.2139
3,2023-01-01 08:00:00,293.73,0.00,7.11,39.34,0.54,4.74,6.23,7.28,1,-102.4628,51.2139
4,2023-01-01 09:00:00,287.06,0.00,6.60,37.91,0.42,4.90,6.56,6.97,1,-102.4628,51.2139
...,...,...,...,...,...,...,...,...,...,...,...,...
508350,2023-03-05 01:00:00,216.96,0.07,0.54,78.68,0.37,0.50,0.53,0.17,2,-128.6108,54.5151
508351,2023-03-05 02:00:00,216.96,0.03,0.80,74.39,0.31,0.50,0.54,0.18,2,-128.6108,54.5151
508352,2023-03-05 03:00:00,216.96,0.00,0.92,71.53,0.27,0.50,0.54,0.18,2,-128.6108,54.5151
508353,2023-03-05 04:00:00,216.96,0.00,0.94,69.38,0.24,0.50,0.54,0.17,2,-128.6108,54.5151


In [ ]:
df.describe()

,co,no,no2,o3,so2,pm2_5,pm10,nh3,aqi,lon,lat
count,508355.000000,508355.000000,508355.000000,508355.000000,508355.000000,508355.000000,508355.000000,508355.000000,508355.000000,508355.000000,508355.000000
mean,306.094317,2.492958,10.925096,55.262630,4.272378,3.677264,4.975740,3.225730,1.623659,-84.825646,47.552628
std,149.628130,12.167109,15.664272,21.437291,7.761484,5.168587,6.968124,5.481733,0.526333,19.773314,3.095620
min,183.580000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,1.000000,-135.056800,42.317400
25%,233.650000,0.000000,1.860000,43.990000,0.640000,0.500000,0.720000,0.280000,1.000000,-100.049400,45.483300
50%,260.350000,0.010000,4.840000,59.370000,1.730000,1.570000,2.270000,1.120000,2.000000,-74.080000,46.670900
75%,313.760000,0.380000,12.510000,70.100000,4.470000,4.590000,6.200000,3.670000,2.000000,-72.316700,49.166600
max,2777.100000,339.750000,178.220000,127.320000,185.010000,104.130000,126.480000,85.120000,5.000000,-52.712800,63.748600


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508355 entries, 0 to 508354
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   dt      508355 non-null  object 
 1   co      508355 non-null  float64
 2   no      508355 non-null  float64
 3   no2     508355 non-null  float64
 4   o3      508355 non-null  float64
 5   so2     508355 non-null  float64
 6   pm2_5   508355 non-null  float64
 7   pm10    508355 non-null  float64
 8   nh3     508355 non-null  float64
 9   aqi     508355 non-null  int64  
 10  lon     508355 non-null  float64
 11  lat     508355 non-null  float64
dtypes: float64(10), int64(1), object(1)
memory usage: 46.5+ MB


In [4]:
spark = SparkSession.builder.appName("AQI_Classification").getOrCreate()

In [5]:
data = spark.read.csv(data, header=True, inferSchema=True)
data

NameError: name 'data' is not defined

In [6]:
data = df.select("co", 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'aqi')

#create a vector assembler to combine the input features

assembler = VectorAssembler(inputCols=["co", 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3'], outputCol='features')
data = assembler.transform(data)

In [7]:
#Split data into training and test set
train_data, test_data = data.randomSplit([0.7, 0.3])

rf = RandomForestClassifier(featuresCol='features', labelCol='aqi')
model = rf.fit(train_data)

23/03/28 23:08:46 WARN MemoryStore: Not enough space to cache rdd_29_0 in memory! (computed 149.9 MiB so far)
23/03/28 23:08:46 WARN BlockManager: Persisting block rdd_29_0 to disk instead.


In [8]:
predictions = model.transform(test_data)

In [10]:
#Evaluate the performance of the model

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='aqi', metricName='accuracy')

# Add additional evaluation metrics
evaluator = evaluator.setMetricName('weightedPrecision').setMetricName('weightedRecall').setMetricName('f1')

# Calculate the evaluation metrics on the test set
accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
weightedPrecision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
weightedRecall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

print("Accuracy = %g" % accuracy)
print("Weighted Precision = %g" % weightedPrecision)
print("Weighted Recall = %g" % weightedRecall)
print("F1 Score = %g" % f1)

Accuracy = 0.87619
Weighted Precision = 0.877763
Weighted Recall = 0.87619
F1 Score = 0.87393
